                    Implementing a simplified version of Locality sensitive hashing

In [1]:
# Sample Sentences
a = "The quick brown fox jumped over the little lazy dog"
b = "An elephant is jumping around in the streets"
c = "You have to be quick, being lazy will take you nowhere"
d = "The little lazy dog jumped over the quick brown fox"

We will be make k shingles with value of k being = 3. For larger datasets with large texts, we have to take the size of k more so that only those texts which are similar get closer vectors.

In [75]:
k = 3
def shingle(text: str, k = 3):
    shingle_list = []
    for i in range(0,len(text) - k):
        shingle_list.append(text[i:i+k])
    return list(set(shingle_list))

In [76]:
# Demonstration of formation of shingles
A = shingle(a,k)
print(A)

['ed ', 'n f', 'itt', 'le ', 'zy ', 'ck ', 'ver', 'ick', 'x j', 'fox', ' do', 'tle', ' qu', 'r t', 'd o', 'ove', 'er ', 'k b', ' li', 'own', 'y d', 'row', 'lit', 'The', ' la', 'ump', ' ju', 'jum', 'ox ', 'qui', 'e l', 'azy', 'bro', ' br', ' ov', 'wn ', 'uic', 'mpe', 'the', 'ttl', 'he ', 'laz', ' th', ' fo', 'e q', 'ped']


In [77]:
import random
random.seed(42)

def vocab(lest : list, k: int):
    newlist = []
    for texts in lest:
        shin = shingle(texts, k)
        for shingl in shin:
            newlist.append(shingl)
    thelist = list(set(newlist))
    random.shuffle(thelist) 
    return thelist


In [78]:
texts = [a,b,c,d]
vocab = vocab(texts, k)
print(vocab)
print("length of vocab is: ", len(vocab))

['k b', 'ein', 'll ', 'ant', ' br', 'hav', 'bei', 'tre', 'ave', 'g j', 'aro', 'in ', 'e q', ' ha', ' ar', 'tak', 'azy', 'row', 'ele', 'he ', 'o b', 'uic', 'n f', ' st', 'mpi', 'e t', 'jum', 'You', 'laz', 'ng ', 'y w', 'ck ', 'und', 'ump', 'le ', ' do', ' wi', 'itt', 'now', 'owh', ' fo', 'lep', 'd i', 'pha', 'dog', 'n e', 'pin', 'wn ', ' qu', 've ', 'e l', 'ou ', 'tle', 'you', 'bro', 'rou', 'is ', 'ck,', 's j', 'be ', 'ke ', 'er ', 'The', 'ttl', 'eph', 'eet', 'ing', ' la', 'nd ', 'ick', 'the', 'fox', ' yo', 'her', 'whe', 'wil', 'str', 'r t', ' no', 'ake', ' ta', 'u n', 'ill', 'ove', ' to', 'g l', 'ed ', 'mpe', 'n t', 'to ', 'l t', ' el', ' ju', 'nt ', ', b', 'y d', ' be', ' li', 'd o', 'An ', 'oun', 'ped', ' th', 'han', 'ree', 'e y', 'k, ', 'g a', 'own', 'x j', 'qui', ' ov', 'u h', 't i', 'og ', ' in', 'e s', 'zy ', ' is', 'ver', 'lit', 'ox ']
length of vocab is:  122


                            Creating One hot vectors for all the text sentences.

In [79]:
one_hot_vectors = []
for text in texts:
    a_1hot = [1 if x in text else 0 for x in vocab]
    one_hot_vectors.append(a_1hot)

print(len(one_hot_vectors))
print(len(one_hot_vectors[0]))

4
122


Here we have 4 122 dimensioned vectors which are one hot encoded for the k shingles. We will now use minhashing to get the different types of values. 

In [80]:
# MINHASHING to get signature
import numpy as np

def findones(arr, lest):
    for i in arr:
        if(lest[i] == 1):
            return int(i)

def get_signature(lest: list, n = 20):
    length = len(lest[0])
    random_matrix = []
    for i in range(n):
        random_matrix.append(np.random.randint(0,length,length))
    signatures = []
    for les in lest:
        sig = np.ones(n)
        for i in range(n):
            sig[i] = findones(random_matrix[i], les)
        signatures.append(sig)
    return signatures

    

In [81]:
g = get_signature(one_hot_vectors)
print(g)

[array([ 62.,  35.,  67.,  44.,  26.,  50.,  35.,  21., 119.,  47.,  21.,
        69.,  33., 120., 101.,  83.,  67.,  83.,   4.,  26.]), array([ 32., 113.,  46.,  68.,  42.,  93.,  65., 102.,  29., 104., 116.,
        23.,  33.,  41.,   3.,  55.,  99.,  93.,   3.,  24.]), array([106.,  38.,  67.,  38.,  73.,   6.,  13., 106.,  39.,  82.,  39.,
        69.,  27.,  51.,  27.,  82.,  67.,  78., 117., 105.]), array([ 62.,  35.,  67., 114.,  26.,  50.,  35.,  21., 119.,  47.,  21.,
        69.,  33., 120., 101.,  83.,  67.,  83.,   4.,  26.])]


In [84]:
# Jaccard function to compare two vectors
def jaccard(a, b):
    ans = 0
    for i in range(len(a)):
        if a[i] in b:
            ans = ans +1
    return ans/len(a)

In [87]:
# Comparing a and b
print(jaccard(shingle(a), shingle(b)), jaccard(g[0], g[1]))
# Comparing a and c
print(jaccard(shingle(a), shingle(c)), jaccard(g[0], g[2]))
# Comparing a and d
print(jaccard(shingle(a), shingle(d)), jaccard(g[0], g[3]))

0.13043478260869565 0.05
0.1956521739130435 0.15
0.9347826086956522 0.95


We have shown that texts which match in their semantics also match in their hashing, so we will use these signatures and Perform an LSH (Locality Sensitiv Hashing on them)

We will make bands of the subvectors and will make candidate pairs if the hasing of the subvecotrs falls into the same bucket

In [88]:
def split_vector(signature, b):
    assert len(signature) % b == 0
    r = int(len(signature) / b)
    # code splitting signature in b parts
    subvecs = []
    for i in range(0, len(signature), r):
        subvecs.append(signature[i : i+r])
    return subvecs

In [90]:
bands = 10
bands_a = split_vector(g[0], bands)
bands_b = split_vector(g[1], bands)
bands_c = split_vector(g[2], bands)
bands_d = split_vector(g[3], bands)

In [91]:
print(bands_a)
print(bands_b)

[array([62., 35.]), array([67., 44.]), array([26., 50.]), array([35., 21.]), array([119.,  47.]), array([21., 69.]), array([ 33., 120.]), array([101.,  83.]), array([67., 83.]), array([ 4., 26.])]
[array([ 32., 113.]), array([46., 68.]), array([42., 93.]), array([ 65., 102.]), array([ 29., 104.]), array([116.,  23.]), array([33., 41.]), array([ 3., 55.]), array([99., 93.]), array([ 3., 24.])]


In [105]:
# def check_bands(banda, bandb):
#     for i in range(len(banda)):
#         equal = 1
#         for j in range(len(banda[i])):
#             if banda[i] != bandb[i]:
#                 equal = 0
#                 break;

#         if banda[i].all() == bandb[i].all():
#             print(banda[i].all())
#             print(bandb[i].all())
#             print("Candidate pairs ", banda[i])
#             return
#     print("Not Candidate pairs")

def check_bands(banda, bandb):
    for i in range(len(banda)):
        if list(banda[i]) == list(bandb[i]):
            print("Candidate pairs ", banda[i])
            return
    print("Not Candidate pairs")
    return

In [108]:
# Checking the candidate pairs of statements
check_bands(bands_a, bands_b)
check_bands(bands_a, bands_c)
check_bands(bands_a, bands_d)
check_bands(bands_b, bands_d)


Not Candidate pairs
Not Candidate pairs
Candidate pairs  [62. 35.]
Not Candidate pairs


Hense, we can see sentence a = "The quick brown fox jumped over the little lazy dog" and sentence 
d = "The little lazy dog jumped over the quick brown fox" are a candidate pair